In [ ]:
! pip install hyperopt
! pip install scikit-learn-intelex

In [1]:
%run search_model.py

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Exception : Too much time, move on to next parameters  
Exception : Too much time, move on to next parameters               
Exception : Too much time, move on to next parameters               
Exception : Too much time, move on to next parameters                            
Exception : Too much time, move on to next parameters                            
Exception : Too much time, move on to next parameters                                
Exception : Too much time, move on to next parameters                                
Exception : Too much time, move on to next parameters                                
Exception : Too much time, move on to next parameters                                
Exception : Too much time, move on to next parameters                               
Exception : Too much time, move on to next parameters                               
Exception : Too much time, move on to next parameters                             
Exception : Too much time, move on to next pa

NameError: name 'count' is not defined

In [2]:
print("Best estimator:", best)

Best estimator: {'bootstrap': 0, 'feature_extractor': 2, 'k': 5, 'max_depth': 0, 'min_samples_leaf': 0, 'min_samples_split': 1, 'n_estimators': 2, 'preprocessor': 0, 'warm_startbool': 1}


1st try : max evals : 100, max runtime : 20, cv: 3

Best estimator (0.6915) : 
feature_extractor= SelectKBest with k=400, 
n_estimators = 100, 
max_depth = None; 
min_samples_split=3, 
min_samples_leaf= 1, 
preprocessor= StandardScaler, 
warm_start = False, 
bootstrap = True 

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X = final_train.values
X_std = StandardScaler().fit_transform(X)
pca = PCA(n_components=50,whiten=True)
pca.fit(X_std)
X_pca = pca.transform(X_std)

In [11]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

cv = cross_validate(estimator=RandomForestRegressor(n_estimators=300,
                                                    max_depth = 20, 
                                                    min_samples_split=2, 
                                                    min_samples_leaf= 2), 
                    X=X_pca, 
                    y=y_train,
                    cv=5, 
                    scoring=["neg_median_absolute_error", "neg_root_mean_squared_error"],
                    n_jobs=-1,
                    verbose=0)

print(f"Mean MAE over 10 folds = {-cv['test_neg_median_absolute_error'].mean():.4f}")
print(f"Mean RMSE over 10 folds = {-cv['test_neg_root_mean_squared_error'].mean():.4f}")

Mean MAE over 10 folds = 0.5595
Mean RMSE over 10 folds = 0.8785
